In [1]:
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import requests
import regex as re
import time
import pickle

In [41]:
#testing
URL = "https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=2&ti=1"
URL2 = "https://texreg.sos.state.tx.us/public/readtac$ext.TacPage?sl=R&app=9&p_dir=&p_rloc=&p_tloc=&p_ploc=&pg=1&p_tac=&ti=1&pt=1&ch=3&rl=1"
page = requests.get(URL2)

soup = BeautifulSoup(page.content, "lxml")

In [42]:
#testing
parts = soup.find_all("a", attrs={"name":True})
[part["href"] for part in parts]


['readtac$ext.TacPage?sl=T&app=9&p_dir=P&p_rloc=204859&p_tloc=&p_ploc=1&pg=2&p_tac=&ti=1&pt=1&ch=3&rl=1',
 'readtac$ext.TacPage?sl=T&app=9&p_dir=N&p_rloc=204859&p_tloc=&p_ploc=1&pg=2&p_tac=&ti=1&pt=1&ch=3&rl=1',
 'readtac$ext.ViewTAC?tac_view=2&ti=1',
 'readtac$ext.ViewTAC?tac_view=3&ti=1&pt=1',
 'readtac$ext.ViewTAC?tac_view=4&ti=1&pt=1&ch=3',
 'readtac$ext.ViewTAC?tac_view=5&ti=1&pt=1&ch=3&sch=A']

In [54]:
def parse_rule_page(r):
    """
    accepts a request response obj for a rule page on the texSOS website and extracts the citation materials.
    returns a dictionary with all relevant information for adding to a pandas df
    """
    #gets filled with source info
    row_dict = {}

    #parses first request
    rule_soup = BeautifulSoup(r.content, 'lxml')

    #handles multi-page rules
    next_page = rule_soup.find_all("a", attrs={"name":"Next"})
    #loops until there are no more "next page" links
    while bool(next_page):
        r = requests.get(URL_base+next_page[0]["href"])
        rule_soup = BeautifulSoup(r.content, 'lxml')
        next_page = rule_soup.find_all("a", attrs={"name":"Next"})

    #link information
    row_dict["URL"]=r.url
    #pandas parses html tables very well
    df_list = pd.read_html(r.text)

    #location info: title, part, subchapter, etc
    cite = df_list[1]
    #source info
    source_note = df_list[2]
    row_dict["Source Note"] = source_note.iloc[-1].values[0]

    #combines the number (part 4, title 16, etc) with the description (administation, rules for alcohol, etc)
    cite["label"] = cite[0]+": "+ cite[1]
    #essentially a case-when for the different types of location info
    for item in cite["label"].values:
        if re.search("^TITLE\s\d+:",item):
            row_dict["Title"]=item
        elif re.search("^PART\s\d+:",item):
            row_dict["Part"]=item
        elif re.search("^CHAPTER\s",item):
            row_dict["Chapter"]=item
        elif re.search("^DIVISION\s", item):
            row_dict["Division"]=item
        elif re.search("^SUBCHAPTER\s", item):
            row_dict["Subchapter"]=item
        elif re.search("^RULE\s§",item):
            row_dict["Rule"]=item
    #returns the completed row as a dictionary
    return row_dict

In [55]:
#create empty dataframe w/ relevant columns
title16_pt4 = pd.DataFrame(columns=["URL", "Title", "Part", "Chapter", "Division", "Subchapter", "Rule", "Source Note"])
#this list will be filled with the information that gets input to the above dataframe
rules = []

#in order to only scrape certain parts/titles you have to add the earlier links to this list so that
# the crawler does not go "up" the tree
#currently the code is configured for part 4 of title 16 -> the links to title 16 and title 16 pt 4 are thus in the list
urls=["https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=3&ti=16&pt=4", "https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=2&ti=16"]

#href in tx sos website are added to the end of this url for the full path
URL_base = "https://texreg.sos.state.tx.us/public/"

#reg ex for a url that is a "rule" page
rule_ex = re.compile("rl=\d+$")
def scrape(site):
    #sleep so I don't get kicked out
    time.sleep(1)

    # getting the request from url
    r = requests.get(site)

    # converting the text
    s = BeautifulSoup(r.text,"lxml")

    #the name == True attr locates only "a" tags with a name attribute, the name attribute exists only in the relevant links
    for i in s.find_all("a", attrs={"name":True}):

    #gets hyperlink
    href = i.attrs['href']

    #creates full url
    site = URL_base+href

    #checks if the link is to a "rule" page with a regex defined above
    match = rule_ex.search(site)
    if match:

    print("Rule " + match.group() + " found")
    #add the site to the url list so it is not scraped again
    urls.append(site)

    #calls parse_rule_page() to get relevant source info and deal with multi-page rules
    rule_dict = parse_rule_page(requests.get(site))
    #append rule information to list of rules
    rules.append(rule_dict)
    #sleep so that texas doesn't think I'm ddos-ing them
    time.sleep(1)


    elif site not in urls:
    #if not a rule page, keep going but don't repeat
    print("scraping " + site)
    urls.append(site)

    # recursive call
    scrape(site)

scraping https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=4&ti=16&pt=4&ch=55
scraping https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=5&ti=16&pt=4&ch=55&sch=A&rl=Y
Rule rl=1 found
Rule rl=10 found
scraping https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=5&ti=16&pt=4&ch=55&sch=B&rl=Y
Rule rl=20 found
Rule rl=30 found
scraping https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=5&ti=16&pt=4&ch=55&sch=C&rl=Y
Rule rl=40 found
scraping https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=5&ti=16&pt=4&ch=55&sch=D&rl=Y
Rule rl=50 found
Rule rl=51 found
Rule rl=52 found
Rule rl=53 found
Rule rl=54 found
Rule rl=55 found
Rule rl=56 found
Rule rl=57 found
Rule rl=58 found
Rule rl=59 found
Rule rl=60 found
Rule rl=61 found
scraping https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=5&ti=16&pt=4&ch=55&sch=E&rl=Y
Rule rl=70 found
Rule rl=71 found
Rule rl=72 found
Rule rl=73 found
Rule rl=74 found
Ru

In [56]:
#testing
URL = "https://texreg.sos.state.tx.us/public/readtac$ext.ViewTAC?tac_view=3&ti=16&pt=4"
scrape(URL)

In [52]:
#testing
title16_pt4 = pd.DataFrame(rules, columns=["URL", "Title", "Part", "Chapter", "Division", "Subchapter", "Rule", "Source Note"])

In [2]:
#this was for saving progress earlier, currently somewhat redundant but keeping around jic
with open('url_list.pkl', 'wb') as f:
    pickle.dump(urls, f)

In [15]:
with open('url_list.pkl', 'rb') as f:
    load_urls = pickle.load(f)

In [4]:
urls_copy = load_urls.copy()

In [5]:
urls_ser = pd.Series(urls_copy)

In [2]:
#also somewhat reduntant testing
urls_ser = urls_ser[urls_ser.str.contains("rl=\d+", regex=True)]

In [29]:
r = requests.get("https://texreg.sos.state.tx.us/public/readtac$ext.TacPage?sl=R&app=9&p_dir=&p_rloc=&p_tloc=&p_ploc=&pg=1&p_tac=&ti=1&pt=15&ch=354&rl=1039")
s = BeautifulSoup(r.content, "lxml")

True

In [24]:
bool(s.find_all("a", attrs={"name":"Next"}))

[<a href="readtac$ext.ViewTAC?tac_view=2&amp;ti=1" name="TITLE">TITLE 1</a>,
 <a href="readtac$ext.ViewTAC?tac_view=3&amp;ti=1&amp;pt=1" name="PART">PART 1</a>,
 <a href="readtac$ext.ViewTAC?tac_view=4&amp;ti=1&amp;pt=1&amp;ch=3" name="CHAPTER">CHAPTER 3</a>,
 <a href="readtac$ext.ViewTAC?tac_view=5&amp;ti=1&amp;pt=1&amp;ch=3&amp;sch=A" name="SUBCHAPTER">SUBCHAPTER A</a>]

In [57]:
s.find_all("a", attrs={"name":re.compile("(TITLE)|(PART)|(CHAPTER)|(DIVISION)|(SUBCHAPTER)")})


str

In [33]:
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[2]
#df[2]= df[0]+": "+ df[1]
df.iloc[-1].values[0]


In [35]:
r = requests.get("https://texreg.sos.state.tx.us/public/readtac$ext.TacPage?sl=R&app=9&p_dir=&p_rloc=&p_tloc=&p_ploc=&pg=1&p_tac=&ti=1&pt=15&ch=354&rl=1037")

parse_rule_page(r)

{'URL': 'https://texreg.sos.state.tx.us/public/readtac$ext.TacPage?sl=R&app=9&p_dir=&p_rloc=&p_tloc=&p_ploc=&pg=1&p_tac=&ti=1&pt=15&ch=354&rl=1037',
 'Source Note': 'Source Note: The provisions of this §354.1037 adopted to be effective June 26, 1997, 22 TexReg 5826; transferred effective September 1, 2001, as published in the Texas Register May 24, 2002, 27 TexReg 4561',
 'Title': 'TITLE 1: ADMINISTRATION',
 'Part': 'PART 15: TEXAS HEALTH AND HUMAN SERVICES COMMISSION',
 'Chapter': 'CHAPTER 354: MEDICAID HEALTH SERVICES',
 'Subchapter': 'SUBCHAPTER A: PURCHASED HEALTH SERVICES',
 'Division': 'DIVISION 3: MEDICAID HOME HEALTH SERVICES',
 'Rule': 'RULE §354.1037: Written Plan of Care'}

In [69]:
title1_rules = pd.DataFrame(columns=["URL", "Title", "Part", "Chapter", "Division", "Subchapter", "Rule", "Source Note"])
row_dict = parse_rule_page(r)


In [73]:
row_dict

{'URL': 'https://texreg.sos.state.tx.us/public/readtac$ext.TacPage?sl=R&app=9&p_dir=&p_rloc=&p_tloc=&p_ploc=&pg=1&p_tac=&ti=1&pt=1&ch=3&rl=7',
 'Source Note': 'Source Note: The provisions of this §3.7 adopted to be effective February 24, 2000, 25 TexReg 1297; amended to be effective September 9, 2001, 26 TexReg 6645; amended to be effective December 16, 2002, 27 TexReg 11729; amended to be effective October 13, 2003, 28 TexReg 8873; amended to be effective June 9, 2004, 29 TexReg 5575; amended to be effective August 6, 2007, 32 TexReg 4747; amended to be effective June 21, 2009, 34 TexReg 3925; amended to be effective May 9, 2012, 37 TexReg 3389; amended to be effective April 27, 2021, 46 TexReg 2709',
 'Title': 'TITLE 1: ADMINISTRATION',
 'Part': 'PART 1: OFFICE OF THE GOVERNOR',
 'Chapter': 'CHAPTER 3: PUBLIC SAFETY OFFICE',
 'Subchapter': 'SUBCHAPTER A: GENERAL GRANT PROGRAM PROVISIONS',
 'Rule': 'RULE §3.7: Selection Process'}

In [91]:
title1_rules

,URL,Title,Part,Chapter,Division,Subchapter,Rule,Source Note
0,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 1: OFFICE OF THE GOVERNOR,CHAPTER 3: PUBLIC SAFETY OFFICE,NaN,SUBCHAPTER A: GENERAL GRANT PROGRAM PROVISIONS,RULE §3.1: Applicability,Source Note: The provisions of this §3.1 adopt...
1,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 1: OFFICE OF THE GOVERNOR,CHAPTER 3: PUBLIC SAFETY OFFICE,NaN,SUBCHAPTER A: GENERAL GRANT PROGRAM PROVISIONS,RULE §3.3: Definitions,Source Note: The provisions of this §3.3 adopt...
2,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 1: OFFICE OF THE GOVERNOR,CHAPTER 3: PUBLIC SAFETY OFFICE,NaN,SUBCHAPTER A: GENERAL GRANT PROGRAM PROVISIONS,RULE §3.5: Submission Process,Source Note: The provisions of this §3.5 adopt...
3,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 1: OFFICE OF THE GOVERNOR,CHAPTER 3: PUBLIC SAFETY OFFICE,NaN,SUBCHAPTER A: GENERAL GRANT PROGRAM PROVISIONS,RULE §3.7: Selection Process,Source Note: The provisions of this §3.7 adopt...
4,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 1: OFFICE OF THE GOVERNOR,CHAPTER 3: PUBLIC SAFETY OFFICE,NaN,SUBCHAPTER A: GENERAL GRANT PROGRAM PROVISIONS,RULE §3.9: Funding Decisions,Source Note: The provisions of this §3.9 adopt...
...,...,...,...,...,...,...,...,...
3962,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 15: TEXAS HEALTH AND HUMAN SERVICES COMMI...,CHAPTER 395: CIVIL RIGHTS,NaN,SUBCHAPTER D: COMPLIANCE MONITORING,RULE §395.31: HHS Agency Compliance,Source Note: The provisions of this §395.31 ad...
3963,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 15: TEXAS HEALTH AND HUMAN SERVICES COMMI...,CHAPTER 395: CIVIL RIGHTS,NaN,SUBCHAPTER D: COMPLIANCE MONITORING,RULE §395.32: Contractor Compliance,Source Note: The provisions of this §395.32 ad...
3964,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 15: TEXAS HEALTH AND HUMAN SERVICES COMMI...,CHAPTER 395: CIVIL RIGHTS,NaN,SUBCHAPTER E: EMPLOYMENT PRACTICES,RULE §395.41: Employment Practices,Source Note: The provisions of this §395.41 ad...
3965,https://texreg.sos.state.tx.us/public/readtac$...,TITLE 1: ADMINISTRATION,PART 15: TEXAS HEALTH AND HUMAN SERVICES COMMI...,CHAPTER 396: EMPLOYEE TRAINING AND EDUCATION,NaN,NaN,RULE §396.1: Purpose and Applicability,Source Note: The provisions of this §396.1 ado...


In [90]:
#title1_rules = pd.DataFrame(columns=["URL", "Title", "Part", "Chapter", "Division", "Subchapter", "Rule", "Source Note"])
for rule_url in urls_ser[2885:].values:
    r = requests.get(rule_url)
    title1_rules = title1_rules.append(parse_rule_page(r), ignore_index=True)
    time.sleep(1)
    

In [93]:
title1_rules.to_excel("title_1_prelim.xlsx")

In [101]:
rl1 = "https://texreg.sos.state.tx.us/public/readtac$ext.TacPage?sl=R&app=9&p_dir=&p_rloc=&p_tloc=&p_ploc=&pg=1&p_tac=&ti=1&pt=1&ch=3&rl=1"

[s for s in rl1.split("&") if re.search("(rl=\d+)|(ch=\d+)|(div=\d+)|(pt=\d+)|(ti=\d+)",s)]

comp = "(rl=\d+)|(ch=\d+)|(div=\d+)|(pt=\d+)|(ti=\d+)"

urls_ser = urls_ser.str.split("&")


In [102]:

urls_ser.apply(lambda x: [s for s in x if re.search("(rl=\d+)|(ch=\d+)|(sch=\d+)|(div=\d+)|(pt=\d+)|(ti=\d+)",s)])


5           [ti=1, pt=1, ch=3, rl=1]
6           [ti=1, pt=1, ch=3, rl=3]
7           [ti=1, pt=1, ch=3, rl=5]
8           [ti=1, pt=1, ch=3, rl=7]
9           [ti=1, pt=1, ch=3, rl=9]
                    ...             
4858    [ti=1, pt=15, ch=395, rl=31]
4859    [ti=1, pt=15, ch=395, rl=32]
4861    [ti=1, pt=15, ch=395, rl=41]
4863     [ti=1, pt=15, ch=396, rl=1]
4864     [ti=1, pt=15, ch=396, rl=2]
Length: 3967, dtype: object

In [ ]:
def convert_to_rowdict(x):
    """
    x: list to be converted into a dictionary that can be fed into a df as a row
    """
    row_dict = {}
    for item in x:
        if re.match("ti=\d+",item):
        elif re.match("pt=\d+",item):
        elif re.match("div=\d+",item):
        elif re.match("rl=\d+",item):
        elif re.match("rl=\d+",item):
                
        
            

In [59]:
#title1 = pd.read_excel("title_1_prelim.xlsx")



Index(['URL'], dtype='object')

In [46]:
title16_pt4

,URL,Title,Part,Chapter,Division,Subchapter,Rule,Source Note


In [60]:
title16_pt4['Source Note'] = title16_pt4['Source Note'].str.split(";")

In [61]:
title16_pt4 = title16_pt4.explode('Source Note')

In [64]:
title16_pt4.to_excel("data/title16_pt4_explode.xlsx", index=False)